In [1]:
from functions.backtesting_functions import *

# Read Files

In [2]:
INPUT_DIRECTORY = './input/sp500_log_returns.rds'

SnP500_lr_r_obj = read_rds(INPUT_DIRECTORY)
SnP500_lr_index_r_obj = SnP500_lr_r_obj['attributes']['index']['data']
SnP500_lr_timestamp_index_r_obj = [datetime.utcfromtimestamp(i).strftime('%Y-%m-%d') for i in SnP500_lr_index_r_obj]
SnP500_lr__data_r_obj = SnP500_lr_r_obj['data']
df_SnP500_lr = pd.DataFrame(data={'Forecast': SnP500_lr__data_r_obj}, index=SnP500_lr_timestamp_index_r_obj)


In [3]:
OUTPUT_DIRECTORY = './output'
FORECASTS_BC_SA = {}

for file in os.listdir(OUTPUT_DIRECTORY):
    if not (file.startswith('.') or file.startswith('b')):
                    
        all_forecasts = []
        for file_in_dir in os.listdir(os.path.join(OUTPUT_DIRECTORY, file)):
            if not file_in_dir.startswith('.'):

                path = os.path.join(OUTPUT_DIRECTORY, file, file_in_dir)
                r_obj = read_rds(path)

                index_r_obj = r_obj['attributes']['index']['data']
                timestamp_index_r_obj = [datetime.utcfromtimestamp(i).strftime('%Y-%m-%d') for i in index_r_obj]
                data_r_obj = r_obj['data']

                df_forecast = pd.DataFrame(data={'Forecast': data_r_obj}, index=timestamp_index_r_obj)
                all_forecasts.append(df_forecast)
        
        df_all_forecasts = pd.concat(all_forecasts).sort_index()
        FORECASTS_BC_SA[file] = df_all_forecasts

In [4]:
FORECASTS_BC_SA.keys()

dict_keys(['arima_base_case', 'arima-egarch_ged_1000', 'arima-garch_base_case', 'arima-sgarch_sged_1000', 'arima-sgarch_ged_500', 'arima-sgarch_sstd_1000', 'arima-sgarch_snorm_1000', 'arima-sgarch_ged_1500'])

### Case 1: ARIMA(p,1,q) 1000 and ARIMA(p,1,q)-SGARCH(1,1) 1000 (Base Case)

In [5]:
case_1 = ['arima_base_case', 'arima-garch_base_case']
df_case_strat_case_1, case_1 = cases_df(FORECASTS_BC_SA, df_SnP500_lr, case_1)
df_case_strat_case_1.to_csv('./output/backtesting_output/equity_lines/case_1.csv')

In [6]:
pm_case_1 = performance_metrics_case_1(case_1[0], case_1[1], df_case_strat_case_1, case_1)
pm_case_1.to_csv('./output/backtesting_output/performance_metrics/pm_case_1.csv')
pm_case_1

,,MAE,MSE,RMSE,MAPE,ARC(%),ASD(%),MD,IR*(%),IR**(%)
,S&P 500,None,None,None,None,6.48,18.77,55.76,34.52,4.01
,arima_base_case,7.72e-03,1.47e-04,1.21e-02,inf,-3.49,18.77,62.76,-18.61,-1.04
,arima-garch_base_case,2.11e+138,1.54e+280,1.24e+140,inf,3.78,18.77,54.23,20.14,1.40


### Case 2: ARIMA(p,1,q) 1000 and ARIMA(p,1,q)-SGARCH(1,1) 1000 and ARIMA(p,1,q)-EGARCH(1,1) 1000

In [7]:
case_2 = ['arima_base_case', 'arima-garch_base_case', 'arima-egarch_ged_1000']
df_case_strat_case_2, case_2 = cases_df(FORECASTS_BC_SA, df_SnP500_lr, case_2)
df_case_strat_case_2.to_csv('./output/backtesting_output/equity_lines/case_2.csv')

In [8]:
pm_case_2 = performance_metrics_case_2(case_2[0], case_2[1], case_2[2], df_case_strat_case_2, case_2)
pm_case_2.to_csv('./output/backtesting_output/performance_metrics/pm_case_2.csv')
pm_case_2

,,MAE,MSE,RMSE,MAPE,ARC(%),ASD(%),MD,IR*(%),IR**(%)
,S&P 500,None,None,None,None,6.48,18.77,55.76,34.52,4.01
,arima_base_case,7.72e-03,1.47e-04,1.21e-02,inf,-3.49,18.77,62.76,-18.61,-1.04
,arima-garch_base_case,2.11e+138,1.54e+280,1.24e+140,inf,3.78,18.77,54.23,20.14,1.40
,arima-egarch_ged_1000,7.57e-03,1.42e-04,1.19e-02,inf,2.45,18.77,42.49,13.04,0.75


### Case 3: ARIMA(p,1,q) 1000 and ARIMA(p,1,q)-SGARCH(1,1) 500 and ARIMA(p,1,q)-SGARCH(1,1) 1000 and ARIMA(p,1,q)-SGARCH(1,1) 1500

In [9]:
case_3 = ['arima_base_case', 'arima-sgarch_ged_500', 'arima-garch_base_case', 'arima-sgarch_ged_1500']
df_case_strat_case_3, case_3 = cases_df(FORECASTS_BC_SA, df_SnP500_lr, case_3)
df_case_strat_case_3.to_csv('./output/backtesting_output/equity_lines/case_3.csv')

In [10]:
df_case_strat_case_3 = df_case_strat_case_3.fillna(1)
pm_case_3 = performance_metrics_case_3__4(case_3[0], case_3[1], case_3[2], case_3[3], df_case_strat_case_3, case_3)
pm_case_3.to_csv('./output/backtesting_output/performance_metrics/pm_case_3.csv')
pm_case_3

,,MAE,MSE,RMSE,MAPE,ARC(%),ASD(%),MD,IR*(%),IR**(%)
,S&P 500,None,None,None,None,6.48,18.77,55.76,34.52,4.01
,arima_base_case,7.72e-03,1.47e-04,1.21e-02,inf,-3.49,18.77,62.76,-18.61,-1.04
,arima-sgarch_ged_500,1.58e-02,8.25e-03,9.08e-02,inf,0.84,24.48,42.41,3.44,0.07
,arima-garch_base_case,2.11e+138,1.54e+280,1.24e+140,inf,3.78,18.77,54.23,20.14,1.40
,arima-sgarch_ged_1500,1.72e-01,1.65e-01,4.07e-01,inf,12.03,483.09,79.38,2.49,0.38


### Case 4: ARIMA 1000 and SGARCH.GED 1000 and SGARCH.SNORM 1000 and SGARCH.SSTD 1000 and SGARCH.SGED 1000

In [11]:
case_4 = ['arima_base_case', 'arima-garch_base_case', 'arima-sgarch_snorm_1000', 'arima-sgarch_sstd_1000', 'arima-sgarch_sged_1000']
df_case_strat_case_4, case_4 = cases_df(FORECASTS_BC_SA, df_SnP500_lr, case_4)
df_case_strat_case_4.to_csv('./output/backtesting_output/equity_lines/case_4.csv')

In [12]:
df_case_strat_case_4 = df_case_strat_case_4.fillna(1)
pm_case_4 = performance_metrics_case_3__4(case_4[0], case_4[1], case_4[2], case_4[3], df_case_strat_case_4, case_4)
pm_case_4.to_csv('./output/backtesting_output/performance_metrics/pm_case_4.csv')
pm_case_4

,,MAE,MSE,RMSE,MAPE,ARC(%),ASD(%),MD,IR*(%),IR**(%)
,S&P 500,None,None,None,None,6.48,18.77,55.76,34.52,4.01
,arima_base_case,7.72e-03,1.47e-04,1.21e-02,inf,-3.49,18.77,62.76,-18.61,-1.04
,arima-garch_base_case,2.11e+138,1.54e+280,1.24e+140,inf,3.78,18.77,54.23,20.14,1.40
,arima-sgarch_snorm_1000,3.06e-02,2.34e-02,1.53e-01,inf,3.88,120.50,44.68,3.22,0.28
,arima-sgarch_sstd_1000,2.59e-02,1.87e-02,1.37e-01,inf,13.03,463.15,79.63,2.81,0.46
